In [1]:
import pandas as pd
import glob

# scienceQA

In [2]:
def read_paqruet(f):
    df = pd.read_parquet(f)
    df["source"] = f
    return df

In [3]:
df = pd.concat([
    read_paqruet(f) for f in glob.glob("additional_data/ScienceQA/*")
])

In [4]:
df["choices"].apply(len).value_counts()

2    11045
3     5078
4     4893
5      192
Name: choices, dtype: int64

In [5]:
df = df[(df["choices"].apply(len) > 2) & (df["image"].isnull())].reset_index(drop=True)

In [6]:
len(df)

3291

In [7]:

def get_choice(ary, i):
    try:
        return ary[i]
    except:
        return None
df["A"] = df["choices"].apply(get_choice, i=0)
df["B"] = df["choices"].apply(get_choice, i=1)
df["C"] = df["choices"].apply(get_choice, i=2)
df["D"] = df["choices"].apply(get_choice, i=3)
df["E"] = df["choices"].apply(get_choice, i=4)

In [8]:
df["prompt"] = df["question"]

In [9]:
df["answer"] = df["answer"].map({0: "A", 1: "B", 2: "C", 3: "D", 4: "E"})

In [10]:
df["prompt"].apply(len).describe()

count    3291.000000
mean       81.677606
std        76.353925
min        15.000000
25%        34.000000
50%        74.000000
75%        94.000000
max       862.000000
Name: prompt, dtype: float64

In [11]:
df[["prompt", "A", "B", "C", "D", "E", "answer", "source"]].to_csv("../data/gpt_generated_data/science_qa.csv", index=False)

# OpenBOok

In [12]:
def read_csv(f):
    df = pd.read_csv(f, sep="\t")
    df["source"] = f
    return df

In [13]:
df = pd.concat([
    read_csv(f) for f in glob.glob("additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/*.tsv")
])

In [14]:
len(df)

5957

In [15]:
df = df[df["Complete Question"].str.contains("\?")].reset_index(drop=True)
len(df)

2115

In [16]:
def get_a(s):
    return s.split("(A) ")[1].split(" (B)")[0]
def get_b(s):
    return s.split("(B) ")[1].split(" (C)")[0]
def get_c(s):
    return s.split("(C) ")[1].split(" (D)")[0]
def get_d(s):
    return s.split("(D) ")[1]

In [17]:
df.columns

Index(['ID', 'Question Stem', 'Choices', 'Complete Question', 'Answer Key',
       'source'],
      dtype='object')

In [18]:
df["prompt"]  = df["Question Stem"]
df["A"] = df["Choices"].apply(get_a)
df["B"] = df["Choices"].apply(get_b)
df["C"] = df["Choices"].apply(get_c)
df["D"] = df["Choices"].apply(get_d)
df["E"] = None
df["answer"] = df["Answer Key"]

In [19]:
df["prompt"].apply(len).describe()

count    2115.000000
mean       56.840662
std        24.898577
min        14.000000
25%        39.000000
50%        53.000000
75%        70.000000
max       197.000000
Name: prompt, dtype: float64

In [20]:
df[["prompt", "A", "B", "C", "D", "E", "answer", "source"]].to_csv("../data/gpt_generated_data/openbook.csv", index=False)

# MMLU

In [21]:
def read_csv(f):
    df = pd.read_csv(f)
    df["source"] = f
    return df

In [22]:
df = pd.concat([
    read_csv(f) for f in glob.glob("additional_data/MMLU/*.csv")
])

In [23]:
len(df)

113884

In [24]:
df = df[df["prompt"].str.contains("\?")]

In [25]:
df["prompt"].apply(len).describe()

count    74972.000000
mean      1363.507243
std        774.422673
min         13.000000
25%        828.000000
50%       1490.000000
75%       1873.000000
max       6527.000000
Name: prompt, dtype: float64

In [26]:
df = df[df["prompt"].apply(len) < 300]
len(df)

10915

In [29]:
df["prompt"].apply(len).describe()

count    10915.000000
mean       107.954283
std         67.283250
min         13.000000
25%         59.000000
50%         86.000000
75%        139.000000
max        299.000000
Name: prompt, dtype: float64

In [27]:
df["E"] = None

In [28]:
df[["prompt", "A", "B", "C", "D", "E", "answer", "source"]].to_csv("../data/gpt_generated_data/mmlu.csv", index=False)